In [31]:
# Written by Chongbin Zhang. 
# Designed to facilitate UCSD HDH's calendar import from events organized in excel table. 

# necessary imports
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime
from icalendar import Calendar, Event

In [35]:
# a cell for helper methods

# stringIsInt
# check if the input string can be parsed into an integer
# if so, return that integer
# else return false
# @param: strin, the input string to be checked

def stringIsInt(strin):
    if type(strin) is not str:
        raise(InputIsNotStringError())
    try:
        intout = int(strin)
        return intout
    except ValueError:
        return False
    

# given a correctly formatted time, return a list with the begin time and (if applicable) end time
# the input has the following format: 11:00am, 1:00pm, or 11:00am-1:00pm
# if there's no dash, the time is begin time, and we assume this event is one hour long
# @param: strin, the input string to be parsed
# the input time is 12-hour, and the output will be converted to 24-hour
def timeProcessor(strin):
    if strin.find('-') == -1:
        pm = False
        begin = strin.split(':') # begin = [hr, min]
        if begin[1].find('p') != -1:
            pm = True
        begin[1] = begin[1][0:2] # truncate am/pm
        begin[0] = stringIsInt(begin[0])
        if pm:
            begin[0] += 12
        begin[1] = stringIsInt(begin[1])
        end = []
        end.append(begin[0] + 1)
        end.append(begin[1])
    else:
        beginpm = False
        endpm = False
        strin = strin.split('-')
        begin = strin[0]
        end = strin[1]
        if begin.find('p') != -1:
            beginpm = True
        if end.find('p') != -1:
            endpm = True
        begin = begin.split(':')
        end = end.split(':')
        begin[1] = begin[1][0:2]
        begin[1] = stringIsInt(begin[1])
        begin[0] = stringIsInt(begin[0])
        if beginpm:
            begin[0] += 12
        end[1] = end[1][0:2]
        end[1] = stringIsInt(end[1])
        end[0] = stringIsInt(end[0])
        if endpm:
            end[0] += 12
    result = []
    if begin[0] == 24:
        begin[0] = 0
    if end[0] == 24:
        end[0] = 0
    result.append(begin[0])
    result.append(begin[1])
    result.append(end[0])
    result.append(end[1])
    return result

In [36]:
# open the excel and parse all the columns into lists
# df stands for data frame
df = pd.read_excel('SampleEvents.xls', sheet_name = 'Sheet1')
listConfName = df['Conference Name']
listHousArea = df['College Housing Area']
listConfCord = df['Conference Coordinator']
listArrvDate = df['Arrival 1 : Date']
listArrTm_Ct = df['Arrival 1 : Time, Count']
listChkInstr = df['Arrival 1 : Check-In Instructions']
listMobi_Plc = df['Mobile Check-In Location  & Date']
    
listArrvTime = []
listArrCount = []

for i in range(0, 4): 
    
    # split arrival time & people count, 0~150 is ERC Desk
    temp = listArrTm_Ct[i].split(', ')
    
    if temp[0].find(':') == -1 and temp[0].find('a') == -1 and temp[0].find('p') == -1: # bad format
        print("ERROR parsing: ", listConfName[i], ", because arrival time count is entered as:“", temp[0], "”Please enter this event manually. ")
        listArrvTime.append(None)
    else:
        listArrvTime.append(timeProcessor(temp[0]))
        
    if len(temp) <= 1:
        listArrCount.append(0)
    else:
        incount = stringIsInt(temp[1])
        if not incount:
            print("ERROR parsing: ", listConfName[i], ", because people count is entered as:“", temp[1], "”. Please enter this event manually. ")
            listArrvTime.append(None)
        else:
            listArrCount.append(incount)

listArrTm_Ct = None # free memory

ERROR parsing:  SKYUS-Senri High School , because arrival time count is entered as:“ Check in time TBD ”Please enter this event manually. 
ERROR parsing:  SKYUS-Senri High School , because people count is entered as:“ 44 Participants including chaperones ”. Please enter this event manually. 
[2, 20, 0]


In [38]:
# Calendar() to write to .ics file
cal = Calendar()

# iterate through the table to create .ics file, 150 is the last for ERC desk in this file
for i in range(0, 4):
    
    if (listArrCount[i] is None or listArrvTime[i] is None):
        i += 1
    
    event = Event()

    peopleCount = listArrCount[i]
    beginhr = listArrvTime[i][0]
    beginmin = listArrvTime[i][1]
    endhr = listArrvTime[i][2]
    endmin = listArrvTime[i][3]
    
    begin = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                     beginhr, beginmin)
    end = datetime(int(listArrvDate[i].year), int(listArrvDate[i].month), int(listArrvDate[i].day), 
                   endhr, endmin)
    
    # Add people count in title
    summary = listConfName[i] + ' (' + str(peopleCount) + ')'
    
    # categorize, "nan" means empty
    if listMobi_Plc[i] == "nan": 
        summary = summary + ' (Check-in)'
    # elif (font is red): 
    #     event.add('categories, 'Check-out')
    else: 
        summary += ' (Mobile Check-in)'
    
    event.add('summary', summary)
    event.add('description', listChkInstr[i])
    event.add('dtstart', begin)
    event.add('dtend', end)
    cal.add_component(event)

    filename = './' + summary + '.ics'
    file = open(filename, 'wb')
    file.write(cal.to_ical())
    file.close()
    event.clear()




IndexError: list index out of range

Extra Credit Challenges: 
I self taught try-catch, iCalendar module and file I/O for this python project. 
· Try-catch is essential when trying to convert an int to string: if it failed it will raise an error, and this   cannot be handled with if-else or other conditional block. To me the most challenging part of try-catch is to
  learn the syntax (since I used to program in Java and C but little in Python) and learning to catch the
  correct type of error. 
· iCalendar and file I/O module is essential for writing the output of all the functions to calendar (.ics)
  file. To me the most challenging part of iCalendar and file I/O is 